In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [6]:
dataset=dataset.drop("User ID",axis=1)

In [7]:
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [8]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [9]:
indep=dataset[["Age","EstimatedSalary","Gender_Male"]]
dep=dataset["Purchased"]

In [10]:
indep.shape

(400, 3)

In [11]:
dep

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
param_grid = {
    'penalty': ['l2'],  
    'solver': ['lbfgs','liblinear','newton-cg','saga'] }  
grid = GridSearchCV(LogisticRegression(), param_grid, refit=True, verbose=3, n_jobs=-1)
grid.fit(indep, dep)
y_pred = grid.predict(indep)
print("Best parameters found: ", grid.best_params_)
print("Best score found: ", grid.best_score_)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


C:\Users\swath\anaconda3\Lib\site-packages\scipy\optimize\_linesearch.py:313: LineSearchWarning: The line search algorithm did not converge
  alpha_star, phi_star, old_fval, derphi_star = scalar_search_wolfe2(
C:\Users\swath\anaconda3\Lib\site-packages\sklearn\utils\optimize.py:74: LineSearchWarning: The line search algorithm did not converge
  ret = line_search_wolfe2(


Best parameters found:  {'penalty': 'l2', 'solver': 'newton-cg'}
Best score found:  0.8225


In [13]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(dep, y_pred)
print(cm)       

[[237  20]
 [ 39 104]]


In [14]:
from sklearn.metrics import classification_report
clf_report = classification_report(dep, y_pred)

In [15]:
print(clf_report)

              precision    recall  f1-score   support

           0       0.86      0.92      0.89       257
           1       0.84      0.73      0.78       143

    accuracy                           0.85       400
   macro avg       0.85      0.82      0.83       400
weighted avg       0.85      0.85      0.85       400



In [16]:
print(cm)

[[237  20]
 [ 39 104]]


In [17]:
grid.predict([[27,57000,0]])

C:\Users\swath\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


array([0], dtype=int64)

In [18]:
re=grid.cv_results_

In [19]:
table=pd.DataFrame.from_dict(re)

In [20]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.103180,0.018227,0.006673,0.005729,l2,lbfgs,"{'penalty': 'l2', 'solver': 'lbfgs'}",0.70,0.9625,0.7625,0.8125,0.7000,0.7875,0.097147,2
1,0.015878,0.003761,0.007338,0.003884,l2,liblinear,"{'penalty': 'l2', 'solver': 'liblinear'}",0.65,0.6500,0.6375,0.6375,0.7500,0.6650,0.042866,3
2,0.354749,0.064013,0.010248,0.006269,l2,newton-cg,"{'penalty': 'l2', 'solver': 'newton-cg'}",0.70,0.9625,0.9375,0.8125,0.7000,0.8225,0.112194,1
3,0.033650,0.006927,0.004592,0.002403,l2,saga,"{'penalty': 'l2', 'solver': 'saga'}",0.65,0.6500,0.6375,0.6375,0.6375,0.6425,0.006124,4
